In [1]:
import os.path as osp
from time import time
from datetime import datetime, timedelta
from functools import partial

import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as T
from torchvision.datasets import MNIST

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

/home/chris/.local/lib/python3.9/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [2]:
class FFNN(nn.Module):
    def __init__(self, in_shape, out_shape, p_d1=0.5, p_d2=0.4, h1=64, h2=32):
        super().__init__()
        fc1 = nn.Linear(in_shape, h1)
        a1  = nn.ReLU()
        d1  = nn.Dropout(p=p_d1)
        fc2 = nn.Linear(h1, h2)
        a2  = nn.ReLU()
        d2  = nn.Dropout(p=p_d2)
        fc3 = nn.Linear(h2, out_shape)
        
        # not applying log_softmax here, as it is applied later in 
        # the torch CCE loss
        
        self.nn = nn.Sequential(fc1, a1, d1, fc2, a2, d2, fc3)

    def forward(self, x):
        x = self.nn(x)
        return x

In [3]:
def train_mnist(config, epochs, checkpoint_dir=None, data_dir=None):
    # create model
    model = FFNN(784, 10, 
                 p_d1=config['p_d1'], 
                 p_d2=config['p_d2'], 
                 h1=config['h1'], 
                 h2=config['h2'])
    
    # load data and make a validation split
    transforms = T.Compose([T.ToTensor(), T.Normalize((0.5,),(0.5)), 
                            T.Lambda(lambda x: torch.flatten(x))])
    dataset_train = MNIST(root='/data/', transform=transforms, train=True)

    train_samples = int(len(dataset_train) * 0.8)
    train_subset, val_subset = random_split(dataset_train,
                                           [train_samples, 
                                            len(dataset_train) - train_samples])
    # create dataloaders
    train_args = {'dataset':train_subset, 
                  'batch_size':config['batch_size'], 
                  'shuffle':True, 
                  'num_workers':8, 
                  'pin_memory':True}
    dataloader_train = torch.utils.data.DataLoader(**train_args)
    val_args  = {'dataset':val_subset, 
                  'batch_size':len(val_subset), 
                  'shuffle':False, 
                  'num_workers':8}
    dataloader_val  = torch.utils.data.DataLoader(**val_args) 
    
    # choose computation host device
    device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = torch.device(device_name)
    model.to(device)
    
    
    optimiser = torch.optim.SGD(params=model.parameters(), lr=config['lr'], momentum=0.9)
    f_loss = nn.CrossEntropyLoss()
    
    # training loop
    for n in range(epochs):
        total_loss = 0.0
        # optimisation
        model.train()
        for idx, (X, y) in enumerate(dataloader_train):
            X, y = X.to(device), y.to(device)
            optimiser.zero_grad()
            y_pred = model(X)
            loss = f_loss(y_pred, y)
            loss.backward()
            total_loss += loss.detach().cpu().item() / len(y) # normalise for batch size
            optimiser.step()
            
        # validation set metrics
        predictions, targets, val_losses = [], [], []
        model.eval()
        # we are adding the metrics tensor for each batch to a list,
        # then concatenating at the end to make one tensor with all samples
        for idx, (X, y) in enumerate(dataloader_val):
            with torch.no_grad():
                y_pred = model(X)
                predictions.append(y_pred.detach())
                targets.append(y)
                val_losses.append(f_loss(y_pred, y).cpu().item())

        predictions = torch.cat(predictions, dim=0)
        targets = torch.cat(targets, dim=0)
        predictions = torch.argmax(F.log_softmax(predictions, dim=1),dim=1)
        corrects = (predictions == targets).sum().item()
        wrongs = len(targets) - corrects
        val_accuracy = corrects / len(targets)
        val_loss = sum(val_losses) / float(len(val_losses))
        
        # save checkpoint
        with tune.checkpoint_dir(n) as checkpoint_dir:
            path = osp.join(checkpoint_dir, 'checkpoint')
            torch.save((model.state_dict(), optimiser.state_dict()), path)
            
        # report metric values back to main scheduler
        tune.report(loss=val_loss, accuracy=val_accuracy)
        
def test_accuracy(model, device='cpu'):
    transforms = T.Compose([T.ToTensor(), T.Normalize((0.5,),(0.5)), 
                            T.Lambda(lambda x: torch.flatten(x))])
    dataset_test  = MNIST(root='/data/', transform=transforms, train=False)
    test_args  = {'dataset':dataset_test, 
                  'batch_size':len(dataset_test), 
                  'shuffle':False, 
                  'num_workers':8}
    dataloader_test  = torch.utils.data.DataLoader(**test_args) 
    
    model.eval()
    predictions, targets = [], []
    with torch.no_grad():
        for (X, y) in dataloader_test:
            y_pred = model(X)
            predictions.append(y_pred.detach())
            targets.append(y)

        predictions = torch.cat(predictions, dim=0)
        targets = torch.cat(targets, dim=0)
        predictions = torch.argmax(F.log_softmax(predictions, dim=1),dim=1)
        corrects = (predictions == targets).sum().item()
        wrongs = len(targets) - corrects
        test_accuracy = corrects / len(targets)
        
    return  test_accuracy

In [4]:
max_epochs = 20
num_samples = 10

config = {'lr':tune.loguniform(1e-4, 1e-1), 
          'batch_size':tune.choice([32, 64, 256, 512]), 
          'p_d1':tune.uniform(0.1,0.9), 
          'p_d2':tune.uniform(0.1,0.9), 
          'h1':tune.choice([32, 64, 256, 512, 1024]), 
          'h2':tune.choice([32, 64, 256, 512, 1024])}


scheduler = ASHAScheduler(metric='loss', 
                        mode='min',
                        max_t=20, 
                        grace_period=2, 
                        reduction_factor=2)

reporter  = CLIReporter(metric_columns=['loss', 'accuracy', 'training_iteration'])

resources = {'cpu':2} 
if torch.cuda.is_available():
    resources['gpu'] = 0.5
    
result = tune.run(partial(train_mnist, epochs=max_epochs),
                  resources_per_trial=resources, 
                  config=config, 
                  num_samples=num_samples, 
                  scheduler=scheduler, 
                  progress_reporter=reporter)

best_trial = result.get_best_trial('loss', 'min', 'last')

2021-07-07 10:15:09,476	INFO services.py:1272 -- View the Ray dashboard at http://127.0.0.1:8265
2021-07-07 10:15:11,306	WARNING experiment.py:293 -- No name detected on trainable. Using DEFAULT.
2021-07-07 10:15:11,307	INFO registry.py:64 -- Detected unknown callable for trainable. Converting to class.
2021-07-07 10:15:11,601	ERROR syncer.py:72 -- Log sync requires rsync to be installed.


== Status ==
Memory usage on this node: 1.9/7.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None
Resources requested: 0/8 CPUs, 0/0 GPUs, 0.0/3.97 GiB heap, 0.0/1.99 GiB objects
Result logdir: /home/chris/ray_results/DEFAULT_2021-07-07_10-15-11
Number of trials: 10/10 (10 PENDING)
+---------------------+----------+-------+--------------+------+------+-------------+----------+----------+
| Trial name          | status   | loc   |   batch_size |   h1 |   h2 |          lr |     p_d1 |     p_d2 |
|---------------------+----------+-------+--------------+------+------+-------------+----------+----------|
| DEFAULT_d4cef_00000 | PENDING  |       |           32 | 1024 |   32 | 0.00397303  | 0.281003 | 0.220526 |
| DEFAULT_d4cef_00001 | PENDING  |       |           32 |  512 |   64 | 0.0199141   | 0.704943 | 0.588271 |
| DEFAULT_d4cef_00002 | PENDING  |       |           64 |  256 | 1024 | 0.000969792 | 0.847073 | 0.5156

(pid=144956) /home/chris/.local/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:174.)
(pid=144956)   return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
(pid=144958) /home/chris/.local/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable be

== Status ==
Memory usage on this node: 2.8/7.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None
Resources requested: 8.0/8 CPUs, 0/0 GPUs, 0.0/3.97 GiB heap, 0.0/1.99 GiB objects
Result logdir: /home/chris/ray_results/DEFAULT_2021-07-07_10-15-11
Number of trials: 10/10 (6 PENDING, 4 RUNNING)
+---------------------+----------+-------+--------------+------+------+-------------+----------+----------+
| Trial name          | status   | loc   |   batch_size |   h1 |   h2 |          lr |     p_d1 |     p_d2 |
|---------------------+----------+-------+--------------+------+------+-------------+----------+----------|
| DEFAULT_d4cef_00000 | RUNNING  |       |           32 | 1024 |   32 | 0.00397303  | 0.281003 | 0.220526 |
| DEFAULT_d4cef_00001 | RUNNING  |       |           32 |  512 |   64 | 0.0199141   | 0.704943 | 0.588271 |
| DEFAULT_d4cef_00002 | RUNNING  |       |           64 |  256 | 1024 | 0.000969792 | 0.847

Result for DEFAULT_d4cef_00003:
  accuracy: 0.7668333333333334
  date: 2021-07-07_10-15-46
  done: false
  experiment_id: e94b76da220347519fa20507605b080b
  hostname: chris-server
  iterations_since_restore: 1
  loss: 0.7905436158180237
  node_ip: 192.168.1.58
  pid: 144960
  should_checkpoint: true
  time_since_restore: 33.733715772628784
  time_this_iter_s: 33.733715772628784
  time_total_s: 33.733715772628784
  timestamp: 1625649346
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d4cef_00003
  
Result for DEFAULT_d4cef_00000:
  accuracy: 0.93275
  date: 2021-07-07_10-15-51
  done: false
  experiment_id: afce06fdab70461f9fc10348aa37716a
  hostname: chris-server
  iterations_since_restore: 1
  loss: 0.2378310114145279
  node_ip: 192.168.1.58
  pid: 144958
  should_checkpoint: true
  time_since_restore: 39.328484535217285
  time_this_iter_s: 39.328484535217285
  time_total_s: 39.328484535217285
  timestamp: 1625649351
  timesteps_since_restore: 0
  training_iteration: 

...

Result for DEFAULT_d4cef_00004:
  accuracy: 0.97625
  date: 2021-07-07_10-25-55
  done: false
  experiment_id: 6f977986f456449abe7fd13205cd4e50
  hostname: chris-server
  iterations_since_restore: 17
  loss: 0.08197546750307083
  node_ip: 192.168.1.58
  pid: 144957
  should_checkpoint: true
  time_since_restore: 579.5874133110046
  time_this_iter_s: 30.230751991271973
  time_total_s: 579.5874133110046
  timestamp: 1625649955
  timesteps_since_restore: 0
  training_iteration: 17
  trial_id: d4cef_00004
  
== Status ==
Memory usage on this node: 2.6/7.7 GiB
Using AsyncHyperBand: num_stopped=7
Bracket: Iter 16.000: -0.09563710168004036 | Iter 8.000: -0.11808866262435913 | Iter 4.000: -0.14761251211166382 | Iter 2.000: -0.5671246349811554
Resources requested: 6.0/8 CPUs, 0/0 GPUs, 0.0/3.97 GiB heap, 0.0/1.99 GiB objects (0.0/2.0 CPU_group_0_095b3f18f9ccb0d439ba4db6cd0d258e, 0.0/2.0 CPU_group_0_9dfef44093f07a6ba130c6d757ccae3f, 0.0/2.0 CPU_group_095b3f18f9ccb0d439ba4db6cd0d258e, 0.0/2.0 CPU

Result for DEFAULT_d4cef_00009:
  accuracy: 0.9730833333333333
  date: 2021-07-07_10-26-20
  done: false
  experiment_id: e361f3100f2b46448367d5bbf6fe0fd4
  hostname: chris-server
  iterations_since_restore: 16
  loss: 0.0907982885837555
  node_ip: 192.168.1.58
  pid: 155580
  should_checkpoint: true
  time_since_restore: 298.87469720840454
  time_this_iter_s: 12.836217403411865
  time_total_s: 298.87469720840454
  timestamp: 1625649980
  timesteps_since_restore: 0
  training_iteration: 16
  trial_id: d4cef_00009
  
Result for DEFAULT_d4cef_00009:
  accuracy: 0.97275
  date: 2021-07-07_10-26-33
  done: false
  experiment_id: e361f3100f2b46448367d5bbf6fe0fd4
  hostname: chris-server
  iterations_since_restore: 17
  loss: 0.0971529632806778
  node_ip: 192.168.1.58
  pid: 155580
  should_checkpoint: true
  time_since_restore: 312.1344618797302
  time_this_iter_s: 13.259764671325684
  time_total_s: 312.1344618797302
  timestamp: 1625649993
  timesteps_since_restore: 0
  training_iteration:

Result for DEFAULT_d4cef_00009:
  accuracy: 0.9740833333333333
  date: 2021-07-07_10-26-59
  done: false
  experiment_id: e361f3100f2b46448367d5bbf6fe0fd4
  hostname: chris-server
  iterations_since_restore: 19
  loss: 0.09059323370456696
  node_ip: 192.168.1.58
  pid: 155580
  should_checkpoint: true
  time_since_restore: 337.9436159133911
  time_this_iter_s: 13.298288106918335
  time_total_s: 337.9436159133911
  timestamp: 1625650019
  timesteps_since_restore: 0
  training_iteration: 19
  trial_id: d4cef_00009
  
== Status ==
Memory usage on this node: 2.3/7.7 GiB
Using AsyncHyperBand: num_stopped=8
Bracket: Iter 16.000: -0.0907982885837555 | Iter 8.000: -0.11808866262435913 | Iter 4.000: -0.14761251211166382 | Iter 2.000: -0.5671246349811554
Resources requested: 4.0/8 CPUs, 0/0 GPUs, 0.0/3.97 GiB heap, 0.0/1.99 GiB objects (0.0/2.0 CPU_group_095b3f18f9ccb0d439ba4db6cd0d258e, 0.0/2.0 CPU_group_0_095b3f18f9ccb0d439ba4db6cd0d258e, 0.0/2.0 CPU_group_072cf17df9fdf3ff5298636eeb895304, 0.0

2021-07-07 10:27:10,265	INFO tune.py:549 -- Total run time: 718.96 seconds (718.80 seconds for the tuning loop).


Result for DEFAULT_d4cef_00009:
  accuracy: 0.9736666666666667
  date: 2021-07-07_10-27-10
  done: true
  experiment_id: e361f3100f2b46448367d5bbf6fe0fd4
  hostname: chris-server
  iterations_since_restore: 20
  loss: 0.08877608180046082
  node_ip: 192.168.1.58
  pid: 155580
  should_checkpoint: true
  time_since_restore: 348.2736442089081
  time_this_iter_s: 10.330028295516968
  time_total_s: 348.2736442089081
  timestamp: 1625650030
  timesteps_since_restore: 0
  training_iteration: 20
  trial_id: d4cef_00009
  
== Status ==
Memory usage on this node: 1.9/7.7 GiB
Using AsyncHyperBand: num_stopped=10
Bracket: Iter 16.000: -0.0907982885837555 | Iter 8.000: -0.11808866262435913 | Iter 4.000: -0.14761251211166382 | Iter 2.000: -0.5671246349811554
Resources requested: 2.0/8 CPUs, 0/0 GPUs, 0.0/3.97 GiB heap, 0.0/1.99 GiB objects (0.0/2.0 CPU_group_0_095b3f18f9ccb0d439ba4db6cd0d258e, 0.0/2.0 CPU_group_095b3f18f9ccb0d439ba4db6cd0d258e, 0.0/2.0 CPU_group_072cf17df9fdf3ff5298636eeb895304, 0.0

In [5]:
print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

Best trial config: {'lr': 0.003973031165244418, 'batch_size': 32, 'p_d1': 0.28100305152407873, 'p_d2': 0.2205255724755377, 'h1': 1024, 'h2': 32}
Best trial final validation loss: 0.07955656945705414
Best trial final validation accuracy: 0.9778333333333333


In [6]:
best_trained_model = FFNN(784, 10, 
                          p_d1=best_trial.config['p_d1'], 
                          p_d2=best_trial.config['p_d2'], h1=best_trial.config['h'])

best_checkpoint_dir = best_trial.checkpoint.value
model_state, optimiser_state = torch.load(osp.join(best_checkpoint_dir, "checkpoint"))
best_trained_model.load_state_dict(model_state)
test_acc = test_accuracy(best_trained_model)
print("Best trial test set accuracy: {}".format(test_acc))

RuntimeError: Error(s) in loading state_dict for FFNN:
	size mismatch for nn.0.weight: copying a param with shape torch.Size([1024, 784]) from checkpoint, the shape in current model is torch.Size([64, 784]).
	size mismatch for nn.0.bias: copying a param with shape torch.Size([1024]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for nn.3.weight: copying a param with shape torch.Size([32, 1024]) from checkpoint, the shape in current model is torch.Size([32, 64]).